In [27]:
import camber

In [28]:
lmp_job = camber.lammps.create_job(
    command="lmp -in unbreakable.lmp",
    engine_size="XSMALL",
    with_gpu=False
)

In [ ]:
lmp_job.status

In [ ]:
lmp_job.read_logs()

In [ ]:
import os
import MDAnalysis as mda
import nglview as nv
import ipywidgets as widgets
from IPython.display import display
import numpy as np

base_path = os.getcwd()
u = mda.Universe(os.path.join(base_path, "trajectory.lammpstrj"), format="LAMMPSDUMP")

try:
    from MDAnalysis.lib.distances import distance_array
    positions = u.atoms.positions
    distances = distance_array(positions, positions)
    bond_pairs = [(i, j) for i in range(len(u.atoms)) for j in range(i+1, len(u.atoms)) if 1.2 <= distances[i, j] <= 1.8]
    if bond_pairs:
        u.add_TopologyAttr('bonds', np.array(bond_pairs))
except:
    try:
        u.atoms.guess_bonds(vdwradii={'1': 2.0}, fudge_factor=0.8)
    except:
        pass

view = nv.show_mdanalysis(u, step=1)
view.clear_representations()

try:
    if len(u.bonds) > 0:
        view.add_representation('ball+stick', selection='all', radius=0.05, bond_radius=0.05, color='hotpink')
    else:
        raise ValueError()
except:
    view.add_representation('licorice', selection='all', radius=0.1, color='hotpink')

play = widgets.Play(value=0, min=0, max=u.trajectory.n_frames-1, step=1, interval=10, description="▶️")
frame_slider = widgets.IntSlider(value=0, min=0, max=u.trajectory.n_frames-1, step=1, description='Frame:')

widgets.jslink((play, 'value'), (view, 'frame'))
widgets.jslink((frame_slider, 'value'), (view, 'frame'))

display(widgets.HBox([play, frame_slider]))
view